In [ ]:
%matplotlib notebook

import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.stats as stats
from design.spectral import Spectral
np.random.seed(0)

multiplier = 1
contour = False
initialization = 'separated_y' #'random', 'separated_x', 'separated_y', 'trivial_x', 'trivial_y', 
Lx = Ly = 1
Nx = 256 
Ny = 64
eps = multiplier*0.03253176201497791
dt, Nt = 0.000001, 100
show_error=False

spec = Spectral(Lx, Ly, Nx, Ny, eps, initialization)

# |k|^4 plot:
plt.imshow(spec.k2**2, cmap=plt.cm.gray)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].set_title("Visualization")
axs[1].set_title("Histogram of relative concentrations")

if(contour):
    axs[0].contour(spec.u)
else:
    image = axs[0].imshow(spec.u, cmap=plt.cm.gray, animated=True)

if show_error:
    error_info = axs[0].text(0.5,0.85, f"Difference: {0}",
        bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
        transform=axs[0].transAxes, ha="center")

histogram = axs[1].hist(spec.u.flatten(), bins=100)

def animate(i):
    spec.advance(dt, Nt)
    if(contour):
        axs[0].cla()
        axs[0].contour(spec.u)
    else:
        image.set_data(spec.u)
    axs[1].cla()
    distribution = spec.u.flatten()

    histogram = axs[1].hist(distribution, bins=50)
    if show_error:
#         err, = [(np.abs(np.sum(spec.u)/(Nx*Ny) - deviation))
        err, = [np.log(np.abs(np.sum(spec.u)/(Nx*Ny)/deviation - 1))/np.log(10)
        if deviation!=0 else -np.inf]
        error_info.set_text(f"Log-error: {err}")
    return 

anim = animation.FuncAnimation(fig=fig, func=animate)

plt.show()